In [90]:
import torchhd
from torchhd.datasets import AirfoilSelfNoise
from torchhd import embeddings

import torch
from torch.utils.data import DataLoader

from sklearn.preprocessing import QuantileTransformer, MaxAbsScaler, MinMaxScaler, StandardScaler

import torchmetrics
from tqdm import tqdm

In [91]:
DIMENSIONS = 2048
NUM_EPOCHS = 10

In [92]:
dataset = AirfoilSelfNoise('../data', download=True)

Files already downloaded and verified


In [93]:
data2 = dataset.data

In [94]:
quantile_transformer = QuantileTransformer(output_distribution="normal", random_state=0)
max_abs_scaler = MaxAbsScaler()
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

In [95]:
dataset.data = quantile_transformer.fit_transform(dataset.data)
dataset.data = max_abs_scaler.fit_transform(dataset.data)
dataset.data = torch.tensor(standard_scaler.fit_transform(dataset.data), dtype=torch.float)
dataset.targets = torch.tensor(min_max_scaler.fit_transform(dataset.targets.view(-1, 1)), dtype=torch.float)

In [96]:
dataset.targets

tensor([[0.6068],
        [0.5802],
        [0.6002],
        ...,
        [0.0857],
        [0.0756],
        [0.0219]])

In [97]:
torch.tensor(dataset.targets.flatten())

/tmp/ipykernel_548/551401381.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(dataset.targets.flatten())


tensor([0.6068, 0.5802, 0.6002,  ..., 0.0857, 0.0756, 0.0219])

In [98]:
dataset.data

tensor([[-0.4287, -1.7622,  1.8686,  1.2794, -0.5631],
        [-0.2795, -1.7622,  1.8686,  1.2794, -0.5631],
        [-0.1398, -1.7622,  1.8686,  1.2794, -0.5631],
        ...,
        [ 0.5901,  0.8498,  0.0259, -0.2622,  1.8525],
        [ 0.7719,  0.8498,  0.0259, -0.2622,  1.8525],
        [ 0.9820,  0.8498,  0.0259, -0.2622,  1.8525]])

In [99]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1)

In [100]:
# item, label = next(iter(train_dataloader))
# print(item)
# print(label)

In [101]:
data2 = QuantileTransformer(output_distribution="normal", random_state=0).fit_transform(dataset.data)
data2 = MaxAbsScaler().fit_transform(data2)
data2 = StandardScaler().fit_transform(data2)
data2

array([[-0.42872327, -1.76217817,  1.86856929,  1.27942626, -0.56306083],
       [-0.27950185, -1.76217817,  1.86856929,  1.27942626, -0.56306083],
       [-0.13983036, -1.76217817,  1.86856929,  1.27942626, -0.56306083],
       ...,
       [ 0.59012637,  0.84977959,  0.02591884, -0.26220202,  1.85252965],
       [ 0.77193837,  0.84977959,  0.02591884, -0.26220202,  1.85252965],
       [ 0.98197709,  0.84977959,  0.02591884, -0.26220202,  1.85252965]])

In [102]:
dataset.data

tensor([[-0.4287, -1.7622,  1.8686,  1.2794, -0.5631],
        [-0.2795, -1.7622,  1.8686,  1.2794, -0.5631],
        [-0.1398, -1.7622,  1.8686,  1.2794, -0.5631],
        ...,
        [ 0.5901,  0.8498,  0.0259, -0.2622,  1.8525],
        [ 0.7719,  0.8498,  0.0259, -0.2622,  1.8525],
        [ 0.9820,  0.8498,  0.0259, -0.2622,  1.8525]])

In [103]:
class LevelHVModel(torch.nn.Module):
    def __init__(self, num_features, num_levels=100, lr=0.00001) -> None:
        super(LevelHVModel, self).__init__()
        
        self.lr = lr
        self.M = torch.zeros(1, DIMENSIONS)
        self.project_sample = embeddings.Projection(num_features, DIMENSIONS)
        self.num_levels = num_levels
        self.project_target = embeddings.Level(num_levels, DIMENSIONS, low=-3, high=3)
        self.level_hypervectors = self.project_target.weight
        
    def encode(self, x):
        sample_hv = self.project_sample(x)
        return torchhd.hard_quantize(sample_hv)
    
    def model_update(self, x, y):
        update = torchhd.bundle(self.M, self.lr * torchhd.bind(self.encode(x), (self.project_target(y))))
        update = update.mean(0)
        self.M = update
        
    def forward(self, x):
        prediciton_hv = torchhd.bind(self.M, torchhd.inverse(self.encode(x)))
        similarities = torchhd.cos_similarity(prediciton_hv, self.level_hypervectors)
        idx = similarities.argmax()
        prediction = torchhd.functional.map_range(idx.to(torch.float), 0, self.num_levels - 1, 0, 1)
        return prediction

In [104]:
model = LevelHVModel(5, num_levels=100, lr=0.0001)

In [105]:
len(train_dataloader.dataset)

1052

In [106]:
test_metric = torchmetrics.MeanSquaredError()

In [107]:
a = torch.tensor([0, 0, 0, 0.])
b = torch.tensor([1, 0, 0, 0.])
c = torch.tensor([1, 0, 1, 0.])

test_metric(a, c)

tensor(0.5000)

In [108]:
model.M

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])

In [109]:
mse_errors = torch.zeros(NUM_EPOCHS)
r2_scores = torch.zeros(NUM_EPOCHS) 

mse = torchmetrics.MeanSquaredError()
r2 = torchmetrics.R2Score()

with torch.no_grad():
    all_predictions = torch.zeros(train_size)
    true_labels = torch.zeros(train_size)
    for epoch in range(NUM_EPOCHS):
        for sample, label in tqdm(train_dataloader, desc="Epoch: {}".format(epoch + 1)):
            model.model_update(sample, label)
        i = 0
        for sample, label in tqdm(train_dataloader, desc="Evaluation: {}".format(epoch + 1)):
            prediction = model(sample)
            all_predictions[i] = prediction
            true_labels[i] = label
            i += 1
        mse_errors[epoch] = mse(all_predictions, true_labels)
        r2_scores[epoch] = r2(all_predictions, true_labels)
            

Evaluation: 10: 100%|██████████| 1052/1052 [00:00<00:00, 1485.27it/s]


In [110]:
model.M

MAP([[ 0.1480,  0.0220, -0.0440,  ..., -0.0140, -0.0720,  0.0140]])

In [111]:
mse_errors

tensor([0.1283, 0.1282, 0.1282, 0.1282, 0.1282, 0.1282, 0.1282, 0.1282, 0.1282,
        0.1282])

In [112]:
r2_scores

tensor([-2.8020, -2.8009, -2.8012, -2.8009, -2.8010, -2.8010, -2.8009, -2.8009,
        -2.8009, -2.8009])

In [113]:
all_predictions

tensor([0.5253, 0.5859, 0.5859,  ..., 0.7374, 0.0808, 0.5859])

In [114]:
true_labels

tensor([0.1934, 0.4144, 0.6675,  ..., 0.7604, 0.5596, 0.5941])